In [1]:
from load_data_json import LoadDataset 
from preprocess import VectorSpaceModel
from preprocess_restart import VectorSpaceModel2
import numpy as np
import time
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds, eigsh
import ipyparallel as ipp
from scipy.sparse import diags
import math


## Load the dataset
Functions to load the dataset and the given queries with associated relevant documents.

In [2]:
dataset = LoadDataset("./data/med/med.json", "./data/med/queries.json", "./data/med/qrels-treceval.txt","med")
# vsm = VectorSpaceModel(dataset.doc_matrix, dataset.name)
vsm = VectorSpaceModel2(dataset.doc_matrix, dataset.name)

c:\Users\dungl\anaconda3\envs\IR_VectorSpaceModel\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
vsm.parallel_lanczos(dataset.query_vectors)

In [ ]:
vsm.parallel_lanczos(dataset.query_vectors, dc=4)